In [1]:
# !gdown 1YiV9ku40vrVigPYNe41AOXZwClKbZ-Oo
# !gdown 1hKTk7XlOEOoJvo65JzYXulsq02O5jSUD
# !gdown 1G1wynkz7l4FlUnjSvLTOupTMom2u-sTL

In [2]:
import pandas as pd
import numpy as np
import json

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression

import os

SEED = 21

In [3]:
PATH = '../data'

In [4]:
df = pd.read_csv(os.path.join(PATH, 'RuMedPrimeData.zip'), sep='\t', compression='zip')
df.head()

,symptoms,anamnesis,icd10,new_patient_id,new_event_id,new_event_time
0,"Сухость кожи, мышечная слабость, нарушение мен...",Месяц назад сильный стресс и на этом фоне появ...,E01.8,qf156c36,q5fc2cb1,2027-05-19
1,"Жалобы ГБ: головная боль, головокружение на фо...","Начало острое, постепенное, после переохлажден...",J06.9,q9321cf8,qe173f20,2023-03-24
2,"головная боль, озноб, насморк, легкая заложенн...","Начало острое, *ДАТА* после переохлаждения. Пр...",J00,qcbc06c5,q653efaa,2026-12-22
3,"На изжога периодически , отрыжка воздухом, . б...",Впервые пояились вышеизложенные жалобы в течен...,K21.0,qfd53b1f,qe48681b,2026-01-15
4,"- лабильность настроения, плаксивость.","Тиреоидэктомия в*ДАТА* по поводу МУЗ, по гисто...",E89.0,q783a54b,q4109cab,2021-05-28


In [5]:
# df2 = pd.read_csv('Итоговая_БАЗА_болезней.csv')

## Данные

In [6]:
df['icd10'] = df['icd10'].apply(lambda s: s.split('.')[0])
df = df[['symptoms', 'icd10']].reset_index(drop=True)

In [7]:
df

,symptoms,icd10
0,"Сухость кожи, мышечная слабость, нарушение мен...",E01
1,"Жалобы ГБ: головная боль, головокружение на фо...",J06
2,"головная боль, озноб, насморк, легкая заложенн...",J00
3,"На изжога периодически , отрыжка воздухом, . б...",K21
4,"- лабильность настроения, плаксивость.",E89
...,...,...
7620,"Насморк,затек слизи по задней стенке.Высыпания...",J30
7621,"на одышку в покое, при незначительной физическ...",J45
7622,на периодическую боль в поясничной области,M54
7623,"Боли в левом плечевом суставе, усилив. в полож...",M19


In [8]:
dict(df['icd10'].value_counts())

{'M54': 734,
 'I11': 360,
 'G54': 304,
 'E06': 220,
 'G90': 215,
 'G44': 171,
 'J06': 162,
 'G20': 151,
 'E66': 136,
 'M13': 116,
 'N76': 112,
 'N41': 108,
 'K29': 107,
 'E04': 95,
 'M19': 90,
 'G40': 85,
 'J30': 81,
 'N20': 78,
 'D25': 76,
 'K81': 75,
 'K21': 72,
 'E11': 70,
 'N30': 70,
 'I67': 67,
 'J45': 65,
 'E01': 65,
 'N95': 65,
 'E05': 58,
 'M02': 58,
 'I10': 56,
 'G52': 55,
 'J00': 55,
 'L30': 54,
 'N83': 52,
 'N80': 50,
 'M17': 50,
 'G98': 48,
 'M53': 48,
 'N97': 45,
 'J35': 44,
 'J18': 43,
 'M75': 43,
 'N40': 42,
 'D50': 41,
 'I20': 39,
 'N86': 39,
 'G25': 39,
 'K80': 38,
 'J31': 38,
 'G43': 38,
 'M51': 37,
 'H81': 37,
 'J42': 35,
 'L40': 34,
 'K58': 34,
 'E89': 34,
 'I49': 33,
 'G62': 32,
 'J84': 32,
 'L20': 32,
 'I83': 31,
 'L21': 30,
 'M05': 30,
 'J01': 30,
 'N81': 30,
 'N84': 30,
 'M77': 30,
 'N39': 29,
 'E03': 29,
 'N93': 28,
 'M15': 28,
 'E22': 28,
 'L70': 27,
 'N91': 27,
 'M42': 26,
 'N28': 25,
 'M06': 25,
 'J20': 24,
 'E28': 24,
 'J44': 23,
 'K26': 23,
 'M10': 22,
 'G

In [9]:
parts = np.split(df.sample(frac=1, random_state=SEED), [int(0.735*len(df)), int(0.8675*len(df))])

In [10]:
train_data = parts[0]
dev_data = parts[1]
test_data = parts[2]

In [11]:
icd_freq = (train_data['icd10'].value_counts() > 10)
labels = [k for k, v in dict(icd_freq).items() if v]

In [15]:
icd_freq

M54     True
I11     True
G54     True
E06     True
G90     True
       ...  
S30    False
I24    False
G53    False
G95    False
F42    False
Name: icd10, Length: 436, dtype: bool

In [12]:
train_data = train_data[train_data['icd10'].isin(labels)]
dev_data = dev_data[dev_data['icd10'].isin(labels)]
test_data = test_data[test_data['icd10'].isin(labels)]

In [13]:
# df_list = [train_data, dev_data, test_data]
# for i in range(0, 3):
#     print(df_list[i].shape)
#     print(df_list[i]['icd10'].value_counts())

## TFIDF и LogReg

In [14]:
def hit_at_n(y_true, y_pred, n=3):
    assert len(y_true) == len(y_pred)
    hit_count = 0
    for l, row in zip(y_true, y_pred):
        order = (np.argsort(row)[::-1])[:n]
        hit_count += int(l in order)
    return round(hit_count / float(len(y_true)) * 100, 2)


def encode_text(tfidf, text_data, labels, l2i, mode='train'):
    if mode == 'train':
        X = tfidf.fit_transform(text_data)
    else:
        X = tfidf.transform(text_data)
    y = labels.map(l2i)
    return X, y


def logits2codes(logits, i2l, n=3):
    codes = []
    for row in logits:
        order = np.argsort(row)[::-1]
        codes.append([i2l[i] for i in order[:n]])
    return codes

In [15]:
text_id = 'symptoms'
label_id = 'icd10' 
index_id = 'idx'

i2l = dict(enumerate(sorted(train_data[label_id].unique())))
l2i = {label: i for i, label in i2l.items()}

In [16]:
tfidf = TfidfVectorizer(analyzer='char', ngram_range=(3, 8))
clf = LogisticRegression(penalty='l2', C=10, multi_class='ovr', n_jobs=-1, verbose=1)

X, y = encode_text(tfidf, train_data[text_id], train_data[label_id], l2i)

In [17]:
# tfidf.vocabulary_

In [20]:
# print(X.shape)
# print(tfidf.get_feature_names_out()[::2000])

In [21]:
X.shape

(4722, 771134)

In [22]:
clf = LogisticRegression(penalty='l2', C=10, multi_class='ovr', n_jobs=-1, verbose=1)

In [23]:
%%time

clf.fit(X, y.values)

[Parallel(n_jobs=-1)]: Using backend LokyBackend with 16 concurrent workers.


RUNNING THE L-BFGS-B CODE

           * * *

Machine precision = 2.220D-16
 N =       771135     M =           10

At X0         0 variables are exactly at the bounds

At iterate    0    f=  3.27304D+03    |proj g|=  2.34900D+03
RUNNING THE L-BFGS-B CODE

           * * *

Machine precision = 2.220D-16
 N =       771135     M =           10

At X0         0 variables are exactly at the bounds
RUNNING THE L-BFGS-B CODE

           * * *

Machine precision = 2.220D-16
 N =       771135     M =           10

At iterate    0    f=  3.27304D+03    |proj g|=  2.34600D+03

At X0         0 variables are exactly at the bounds

At iterate    0    f=  3.27304D+03    |proj g|=  2.34900D+03


 This problem is unconstrained.
 This problem is unconstrained.
 This problem is unconstrained.


RUNNING THE L-BFGS-B CODE

           * * *

Machine precision = 2.220D-16
 N =       771135     M =           10

At X0         0 variables are exactly at the bounds

At iterate    0    f=  3.27304D+03    |proj g|=  2.19600D+03
RUNNING THE L-BFGS-B CODE

           * * *

Machine precision = 2.220D-16
 N =       771135     M =           10

At X0         0 variables are exactly at the bounds

At iterate    0    f=  3.27304D+03    |proj g|=  2.30500D+03
RUNNING THE L-BFGS-B CODE

           * * *

Machine precision = 2.220D-16
 N =       771135     M =           10

At X0         0 variables are exactly at the bounds

At iterate    0    f=  3.27304D+03    |proj g|=  2.31100D+03
RUNNING THE L-BFGS-B CODE

           * * *

Machine precision = 2.220D-16
 N =       771135     M =           10
RUNNING THE L-BFGS-B CODE

           * * *

Machine precision = 2.220D-16
 N =       771135     M =           10

At X0         0 variables are exactly at the bounds

At X0         0 variables are e

 This problem is unconstrained.
 This problem is unconstrained.
 This problem is unconstrained.
 This problem is unconstrained.
 This problem is unconstrained.
 This problem is unconstrained.
 This problem is unconstrained.
 This problem is unconstrained.
 This problem is unconstrained.


...

 This problem is unconstrained.



           * * *

Tit   = total number of iterations
Tnf   = total number of function evaluations
Tnint = total number of segments explored during Cauchy searches
Skip  = number of BFGS updates skipped
Nact  = number of active bounds at final generalized Cauchy point
Projg = norm of the final projected gradient
F     = final function value

           * * *

   N    Tit     Tnf  Tnint  Skip  Nact     Projg        F
*****     43     46      1     0     0   7.139D-05   9.507D+01
  F =   95.068372982460488     

CONVERGENCE: NORM_OF_PROJECTED_GRADIENT_<=_PGTOL            

           * * *

Tit   = total number of iterations
Tnf   = total number of function evaluations
Tnint = total number of segments explored during Cauchy searches
Skip  = number of BFGS updates skipped
Nact  = number of active bounds at final generalized Cauchy point
Projg = norm of the final projected gradient
F     = final function value

           * * *

   N    Tit     Tnf  Tnint  Skip  Nact     Projg        F
***

[Parallel(n_jobs=-1)]: Done 108 out of 108 | elapsed:  3.5min finished


CPU times: user 1.92 s, sys: 1.03 s, total: 2.95 s
Wall time: 3min 29s


LogisticRegression(C=10, multi_class='ovr', n_jobs=-1, verbose=1)

In [24]:
X_val, y_val = encode_text(tfidf, dev_data[text_id], dev_data[label_id], l2i, mode='val')
y_val_pred = clf.predict_proba(X_val)

accuracy = hit_at_n(y_val, y_val_pred, n=1)
hit_3 = hit_at_n(y_val, y_val_pred, n=3)
print (f'\nscores on dev set: {accuracy} / {hit_3}')


scores on dev set: 49.04 / 74.4


In [1]:
y_val

NameError: name 'y_val' is not defined

In [25]:
X_test, _ = encode_text(tfidf, test_data[text_id], test_data[label_id], l2i, mode='test')
y_test_pred = clf.predict_proba(X_test)

accuracy = hit_at_n(_, y_test_pred, n=1)
hit_3 = hit_at_n(_, y_test_pred, n=3)
print (f'\nscores on test set: {accuracy} / {hit_3}')

test_codes = logits2codes(y_test_pred, i2l)

recs = []
for true, pred in zip(test_data[label_id], test_codes):
    recs.append({label_id: true, 'prediction': pred})


scores on test set: 49.29 / 73.94


/home/dima/anaconda3/envs/torch/lib/python3.8/site-packages/sklearn/linear_model/_logistic.py:444: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


In [ ]:
out_fname = 'results.jsonl'
with open(out_fname, 'w') as fw:
    for rec in recs:
        json.dump(rec, fw, ensure_ascii=False)
        fw.write('\n')